**TUGAS**

In [41]:
import tensorflow as tf
import numpy as np
import os
import time

In [42]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')



In [43]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [44]:

# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [45]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [46]:
# Print unique characters
for char in vocab:
    print(char, end=' ')


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [47]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [48]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [49]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [50]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [51]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [52]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [53]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [54]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [55]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [56]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [57]:
seq_length = 100


In [58]:
# Metode batch dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [59]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [60]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [61]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [62]:
dataset = sequences.map(split_input_target)

In [63]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [64]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [65]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [66]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [67]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [68]:
#uji model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [69]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [71]:
sampled_indices

array([ 9,  5, 51, 15, 62, 37, 48, 59, 40,  8, 39,  7, 50,  1, 24,  2, 34,
       26, 37, 40,  8, 11, 19, 14, 44, 32,  4, 24, 49, 43, 14,  2, 39, 23,
       37, 61, 19, 24,  1, 17, 65, 43, 10, 19,  8, 26, 26, 54, 61,  1, 33,
       26,  1,  9, 65,  8, 49,  1, 32, 52,  9, 55, 46, 44,  3, 29, 21, 59,
        0, 63, 37, 27, 51, 20,  4, 14, 15, 61, 36,  3, 51, 29, 17, 45, 24,
       62, 42, 26, 18,  2, 44, 11, 16,  4, 59, 62, 42, 54, 63, 33])

In [72]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"r are your eyebrows?\n\nFirst Lady:\nBlue, my lord.\n\nMAMILLIUS:\nNay, that's a mock: I have seen a lady'"

Next Char Predictions:
 b'.&lBwXita-Z,k\nK UMXa-:FAeS$KjdA ZJXvFK\nDzd3F-MMov\nTM\n.z-j\nSm.pge!PHt[UNK]xXNlG$ABvW!lPDfKwcME e:C$twcoxT'


In [73]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [74]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190182, shape=(), dtype=float32)


In [75]:
tf.exp(example_batch_mean_loss).numpy()

66.03482

In [76]:
model.compile(optimizer='adam', loss=loss)

In [77]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [78]:
EPOCHS = 10

In [79]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 15s 57ms/step - loss: 2.7182
Epoch 2/10
172/172 [==============================] - 16s 64ms/step - loss: 1.9884
Epoch 3/10
172/172 [==============================] - 14s 57ms/step - loss: 1.7132
Epoch 4/10
172/172 [==============================] - 12s 55ms/step - loss: 1.5513
Epoch 5/10
172/172 [==============================] - 13s 57ms/step - loss: 1.4522
Epoch 6/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3834
Epoch 7/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3302
Epoch 8/10
172/172 [==============================] - 12s 56ms/step - loss: 1.2848
Epoch 9/10
172/172 [==============================] - 12s 58ms/step - loss: 1.2431
Epoch 10/10
172/172 [==============================] - 11s 57ms/step - loss: 1.2039


In [80]:
# membuat prediksi satu langkah:


class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [81]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [82]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ROMEO:
Adieu, good stay, pur, heaven, yields; out against me.

LEONTES:
Now, sir, if not his
undergrance!
Must be pureful: our fiest and falsehood
God know thee and a black is for the gent.

PAULINA:
Why, I am crame
For beauty and selest upon your heom
of such a reash.

CAMILLO:
Alack, be it son, but for thou namest
You. in for me, help! it shall see we will;
Who in the pall hour mighty straight at once
a kind? she'll brain, Lewis camebully.

MENENIUS:
Why, touch choses suitoward out the other,
Whose hopel would slew thou one than Do ask
Our condxict of abuses: they could cannot speak
Well well may keep you affaits from him on simples, and a cloin-

GONZALO:
Wherefor I, in good lord,
But now it are for menine
That your presentuptier conterpets died too
Creward, look'd and roint me for her own,
And so you consent in my brother
You that knew makes Buleasing pritceen plain,
For 'twould I'll prove thee all, that ne'er dismistues,
Whose souls o'er last Auftley's life at that deight you.

QU

In [83]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nGive me your table, combary.\n\nBAPOT:\nGood friends than spood up almost soul.\n\nPAMIRL:\nMy lord,\nMy husband lies thee tears.\n\nJOHN OF GAUNT:\nNo, by 'I am speak.\n\nThore:\nOut, I that same you.\n\nLUCIO:\n\nHORTENSIO:\n\nROMEO:\nMy heart with showshreps to you speak;\nAnd thou wilt did usurp on us.\n\nSecond Gentleman:\nI will be counted's year;\nFor I will bring me or truth more reason\nWhen I swear that thou condution will keep thee\nSlugh on to be duty fit the king his spain-forw'd\nAnd sen in thy kinsman.'\n\nGLOMAS:\nJoy, callet! I will ent\nTo supple my led tcan nature down.\n\nBENVOLIO:\nSuch a lematious peace:\nMy woothy valave, and, widow, as to\nWere but up, such distor miserable.\n\nBRUTUS:\nAnd if not marry to accussems\nFor ne'er last thoushnow to conquer while!\n\nGLOUCESTER:\nBe you, for ortague, we will up.\n\nHERMIONE:\nCome, come, I bedow you then,\nHis acting with this Fortules unshape,\nAnd bellats as hand and happy and woo'd.\n\nFirst 

In [84]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [85]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

In [86]:
for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Go, cross, the law may out op his wife,
What a youd love tell her and till now
Me that would till h


In [87]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [88]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [89]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [90]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 55ms/step - loss: 2.7341


In [91]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1796
Epoch 1 Batch 50 Loss 2.0371
Epoch 1 Batch 100 Loss 1.9622
Epoch 1 Batch 150 Loss 1.8575

Epoch 1 Loss: 2.0017
Time taken for 1 epoch 14.63 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8731
Epoch 2 Batch 50 Loss 1.7665
Epoch 2 Batch 100 Loss 1.6976
Epoch 2 Batch 150 Loss 1.6623

Epoch 2 Loss: 1.7242
Time taken for 1 epoch 20.47 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5951
Epoch 3 Batch 50 Loss 1.5771
Epoch 3 Batch 100 Loss 1.5097
Epoch 3 Batch 150 Loss 1.5145

Epoch 3 Loss: 1.5628
Time taken for 1 epoch 12.42 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4607
Epoch 4 Batch 50 Loss 1.4585
Epoch 4 Batch 100 Loss 1.4824
Epoch 4 Batch 150 Loss 1.4782

Epoch 4 Loss: 1.4617
Time taken for 1 epoch 20.47 sec
_____________________________________________________________________

**Soal**

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

**Jawab**
Pada praktikum 2, pendekatan pelatihan yang digunakan lebih sederhana dan umum, yaitu dengan model.fit. Pada tugas, pendekatan pelatihan yang digunakan lebih spesifik dan kompleks, dengan kustomisasi tertentu. Dalam pendekatan ini, metode train_step didefinisikan dalam model turunan untuk mengatur pelatihan pada tingkat batch. Secara eksplisit dilakukan perhitungan loss, gradien, dan penerapan pembaruan bobot model dengan apply_gradients. Objek tf.metrics.Mean juga digunakan untuk menghitung rata-rata loss selama pelatihan. Pendekatan ini memberikan lebih banyak kontrol dan fleksibilitas dalam pengaturan pelatihan model.